<a href="https://colab.research.google.com/github/aruaru0/kaggle_h_and_m/blob/main/%E5%8D%94%E8%AA%BF%E3%83%95%E3%82%A3%E3%83%AB%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import multiprocessing
multiprocessing.cpu_count()

4

In [3]:
!pip install --upgrade implicit 

     |████████████████████████████████| 18.5 MB 5.9 MB/s 


In [4]:
!pip install optuna

     |████████████████████████████████| 308 kB 9.8 MB/s 
     |████████████████████████████████| 81 kB 11.0 MB/s 
     |████████████████████████████████| 210 kB 69.9 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 146 kB 90.7 MB/s 
     |████████████████████████████████| 113 kB 90.9 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=eb1e1df401e7f4fd5bd32b29a831b1523b9b2778a133c4749ba6bd4a6ddf9197
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:
#初期の必要なライブラリをインポート
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k
import optuna

In [6]:
base_path = '/content/drive/MyDrive/datas/h_and_m/'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(csv_sub)
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

In [7]:
df['t_dat']

0          2018-09-20
1          2018-09-20
2          2018-09-20
3          2018-09-20
4          2018-09-20
              ...    
31788319   2020-09-22
31788320   2020-09-22
31788321   2020-09-22
31788322   2020-09-22
31788323   2020-09-22
Name: t_dat, Length: 31788324, dtype: datetime64[ns]

In [8]:
# """
# Trying with less data(少ないデータで試す)
# """

# #df = df[df['t_dat'] > '2020-08-21']
# df = df[df['t_dat'] > '2020-06-21']
# df.shape

In [9]:
#s0 =  ('2019-09-01' <= df['t_dat']) & (df['t_dat'] < '2019-10-10') 
s1 = df['t_dat'] > '2020-08-01'
df = df[s1]
df.shape

(1993503, 5)

In [10]:
validation_cut = df['t_dat'].max() - pd.Timedelta(7,"d")

# dfの最新の日付から、７日分を差し引き、
# validation_cutより大きい日付は検証用（１週間）
# 　　　　　　　　　 小さい場合は学習用（３週間）とします。


In [11]:
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
29794821,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0853474004,0.033881,2
29794822,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865594003,0.013542,2
29794823,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865699001,0.025407,2
29794824,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0822959001,0.050831,2
29794825,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0888024005,0.084729,2


In [12]:
ALL_USERS = dfu['customer_id'].unique().tolist()
ALL_ITEMS = dfi['article_id'].unique().tolist()

#上では、それぞれのカスタマー名と商品名の固有の値を抽出させ、リストに変換させている

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))
"""
enumerate()を用いることにより、インデックス番号, 要素と取得することが出来る。
商品、アイテム名を番号処理することが可能。
"""

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

df['user_id'] = df['customer_id'].map(user_map)
df['item_id'] = df['article_id'].map(item_map)

del dfu, dfi

In [13]:
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,user_id,item_id
29794821,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0853474004,0.033881,2,38,92348
29794822,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865594003,0.013542,2,38,95146
29794823,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865699001,0.025407,2,38,95177
29794824,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0822959001,0.050831,2,38,86201
29794825,2020-08-02,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0888024005,0.084729,2,38,100027


In [14]:
def to_user_item_coo(df):
    """
    Turn a dataframe with transactions into a COO sparse items x users matrix
    トランザクションを含むデータフレームをCOOスパースアイテム×ユーザー行列に変換する
    """
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo

In [15]:
def split_data(df, validation_days=7):
    validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days,"d")
    df_train = df[df['t_dat'] < validation_cut]
    df_val = df[df['t_dat'] >= validation_cut]
    return df_train, df_val

In [16]:
"""
Split into training and validation and create various matrices
    トレーニング用と検証用に分割し、各種マトリクスを作成する
    
        
         Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
             csr_train: training data in CSR sparse format and as (users x items)
             csr_val:  validation data in CSR sparse format and as (users x items)
作成した学習用データと検証用データに分け、それぞれをCOOスパースアイテム×ユーザー行列に変換する
その後、作成したスパース行列を```tocsr```を使って、データをcsrに変換している。
"""
def get_val_matrices(df, validation_days=7):
    df_train, df_val = split_data(df, validation_days=validation_days)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }

In [17]:
def validate(matrices, factors=200, iterations=20, regularization=0.01, 
show_progress=True):
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42,
                                                 use_gpu=True)
    # factors:    計算する潜在的な要因の数
    # iterations: データをフィットする際に使用する ALS の反復回数
    # regularization: 使用する正則化係数(L1正則化)
    
    model.fit(coo_train, show_progress=show_progress)
    #show_progress 学習の進捗を確認
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [18]:
matrices = get_val_matrices(df)

In [ ]:
# %%time
# best_map12 = 0
# for factors in [10, 25, 50, 75, 100]:
#     for iterations in [3, 12, 14, 15, 20]:
#         for regularization in [0.01]:
#             map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
#             if map12 > best_map12: 
#                 best_map12 = map12
#                 best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
#                 print(f"Best MAP@12 found. Updating: {best_params}")

## Train

In [ ]:
%%time
def objective(trial):
  factors = trial.suggest_int('factors', 1, 500, step=1, log=False)
  iterations = trial.suggest_int('iterations', 1, 50, step=1, log=False)
  regularization = trial.suggest_float('regularization', 0.01, 0.2, step=0.001)
  map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
  return -map12

study = optuna.create_study()
study.optimize(objective, n_trials=1000)

## Make Submission

In [19]:
best_params = study.best_params
best_params

NameError: ignored

In [20]:
# {'factors': 239, 'iterations': 5, 'regularization': 0.078}
best_params = {'factors': 239, 'iterations': 5, 'regularization': 0.078}

In [21]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)

    return model

In [22]:
from collections import defaultdict

def remove_tmp(in_df) :
  m = defaultdict(int)
  target, tmax = "", 0
  for i in tqdm(range(len(in_df))) :
    p = in_df['prediction'][i]
    m[p] += 1
    if m[p] > tmax :
      tmax = m[p]
      target = p

  l = []
  for i in tqdm(range(len(in_df))) :
    p = in_df['prediction'][i]
    if p == target :
      l.append("")
    else :
      l.append(p)
  return l

In [45]:
from tqdm import tqdm

th = 0.001

def submit(model, csr_train, submission_name="submissions.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    # for startidx in tqdm(range(0, len(to_generate), batch_size)):
    #     batch = to_generate[startidx : startidx + batch_size]
    #     ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
    #     for i, userid in enumerate(batch):
    #         customer_id = user_ids[userid]
    #         user_items = ids[i]
    #         article_ids = [item_ids[item_id] for item_id in user_items]
    #         preds.append((customer_id, ' '.join(article_ids)))

    for startidx in tqdm(range(0, len(to_generate), batch_size)):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = []
            for id, sc in zip(ids[i], scores[i]) :
              if sc < th : continue
              user_items.append(id)
            #user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))


    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])

    # df_preds['prediction'] = remove_tmp(df_preds)

    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [46]:
matrices

{'coo_train': <1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
 	with 1727139 stored elements in COOrdinate format>,
 'csr_train': <1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
 	with 1525847 stored elements in Compressed Sparse Row format>,
 'csr_val': <1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
 	with 236622 stored elements in Compressed Sparse Row format>}

In [47]:
#matrices
coo_train = matrices['coo_train']
csr_train = coo_train.tocsr()

model = train(coo_train, **best_params)

  0%|          | 0/5 [00:00<?, ?it/s]

In [48]:
%%time
df_preds = submit(model, csr_train, submission_name="cf104.csv");

100%|██████████| 686/686 [00:48<00:00, 14.23it/s]


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0762846031 0568601006 0568601044 0568601007 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0791587015 0791587001 07...


(1371980, 2)
CPU times: user 51.7 s, sys: 997 ms, total: 52.7 s
Wall time: 52.2 s


In [50]:
coo_train = to_user_item_coo(df)
csr_train = coo_train.tocsr()

model = train(coo_train, **best_params)

  0%|          | 0/5 [00:00<?, ?it/s]

In [52]:
%%time
df_preds = submit(model, csr_train, submission_name="cf.csv");

100%|██████████| 686/686 [00:49<00:00, 13.86it/s]


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0762846031 0568601006 0568601044 0568601007 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0805000001 0794321011 0794321007 0740519002 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0791587015 0791587018 07...


(1371980, 2)
CPU times: user 53.5 s, sys: 463 ms, total: 53.9 s
Wall time: 53.5 s


In [53]:
!cp cf*.csv /content/drive/MyDrive/datas/h_and_m